<a href="https://colab.research.google.com/github/sdlanda77/portfolio/blob/main/Sam_L_hw2_release.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem 1 (30 points)

Run or download the latest version (4.6.1) of the "Consistency based CSP solver" applet from [AIspace](http://www.aispace.org/downloads.shtml). Then navigate to "File" at the top menu, followed by "Load Sample CSP", and select "Scheduling Problem 1". 

1. Give an explicit form of the constraint between variables B and E.

2. Express the three binary constraints between variables B, C, and E as one trinary constraint (either implicit or explicit).

3. Suppose we are doing backtracking search and we have not yet assigned any variables. Which variable(s) should we assign first based on the MRV heuristic? Alternatively, which variable(s) should we assign first based on the degree heuristic?

4. Suppose we assign E to the value 2 from the beginning. List the variables and corresponding domains that get changed as a result of forward checking. 

5. What happens if we enforce arc consistency for the entire CSP after assigning E = 2? (You should do this by hand first, and then use the applet to check your work if you wish.) Describe, step by step, the consequences on each variable's domain as a result of the initial assignment that gets us to the observed result.


1. C = {(B,E) ∈ {(2,1), (4,1), (4,2), (4,3)}} 

2. C = {(B,C,E) ∈ {(2,3,1), (2,4,1), (4,3,2), (4,3,1)}}

3. Based on the MRV heruistic, we should assign B or C first, becasue they have the fewest legal values. Based on the degree heuristic, we should assign B or E first, since they are involved in the greatest number of constraints on other variables. 

4. Domain of A = {3,4}

  Domain of B = {4}

  Domain of C = {3,4}

  Domain of D = {3,4} 

5. 1. The values of the variables listed above are reduced like shown above. 

  2. Due to the constraint C < D, domain of D = {4} and domain of C ={3}.

  3. Due to the constraint not(A=B), domain of A = {3}.

  4. Due to the constraint A=D, domain of A = domain of D = {}.

  5. Due to the constraint E < A, domain of E = {}.

  6. Due to the constraint E < B, domain of B = {}.

  7. Due to the constraint E < C, domain of C = {}.

  All variables domain are the empty set. 

# *(m,n,k)*-Game

In this assignment you will design and play with a simple agent for a [*m,n,k*-game](https://en.wikipedia.org/wiki/M,n,k-game). This is a generalization of tic-tac-toe to a *m* x *n* board, with the goal of getting *k* in a row. As in tic-tac-toe, there are two players, X and O, and they take turns marking cells on the board. We make the following assumptions:
*   This is a zero-sum game; a win for X is equally a loss for O and vice versa. We will take X to be the maximizing player and O to be the minimizing player.
*   Unless otherwise specified, X always goes first.

Interestingly, X will never lose if it plays perfectly, as the first player always has an advantage in *m,n,k* (see the Wikipedia article for a simple argument). We will come back to this point at the end of the assignment. 

You'll find the game-specific functions ```actions```, ```result```, and ```terminal``` used by minimax and alpha-beta search below. The last one also calls the utility functions ```k_in_row``` and ```sequences```. Here are the details...
*   States are represented as 2d numpy arrays. Elements may be 'X', 'O', or '.' for blanks.
*   Taking an action means picking a blank space and marking it. ```actions``` returns all possible actions, or equivalently, all indices of blanks.
*   ```result``` creates and returns a new state with the indicated space filled in by the given player (either 'X' or 'O').
* ```terminal``` serves as both a terminal test and utility function to save some work. First return argument is True (if someone has won or no blanks are left) or False; second argument is utility if the state is terminal or ```None``` otherwise.

In [ ]:
import numpy as np

def actions(state):
  # Returns indices of all blank spaces on board
  return [i for i,s in np.ndenumerate(state) if s=='.']

def result(state, player, action):
  # Returns a new state (deepcopied) with action space taken by player
  new_state = state.copy()
  new_state[action] = player
  return new_state

def terminal(state, k):
  # Test whether state is a terminal or not; also return game score if yes
  blanks = np.count_nonzero(state == '.')
  X_indices = [i for i,s in np.ndenumerate(state) if s=='X']
  if k_in_row(X_indices, k): 
    return True, 1+blanks
  O_indices = [i for i,s in np.ndenumerate(state) if s=='O']
  if k_in_row(O_indices, k): 
    return True, -(1+blanks)
  if blanks == 0: 
    return True, 0
  return False, None

#-------------------------------------------------------------------------------
# Utility functions used by terminal (above)

def k_in_row(indices, k):
  # Test whether there are k consecutive indices in a row in the given list
  index_set = set(indices)
  for i in indices:
    for seq in sequences(i, k):
      if seq.issubset(index_set):
        return True
  return False

def sequences(i, k):
  # Return 4 sets of k indices in the "rows" starting from index i
  across = set([(i[0], i[1]+j) for j in range(k)])
  down = set([(i[0]+j, i[1]) for j in range(k)])
  diagdown = set([(i[0]+j, i[1]+j) for j in range(k)])
  diagup = set([(i[0]+j, i[1]-j) for j in range(k)])
  return across, down, diagdown, diagup

## Response 1 (5 points)

Why does the utility computation in ```terminal``` include the number of blanks left in a terminal state? Note that we could have gone with the standard $\pm1$ utility for a X win/loss as well, but the extra "bonus" provides some useful information.

Players want to maximize their own utiltiy. If there are more blank spaces in a terminal state, that means that we arrived at the solution sooner, which means it will be easier to search for and therefore take fewer moves. It is therefore a better solution if the utility is higher, which is why we want to know the number of blank spaces left: higher utility at a terminal state means we can reach the solution in fewer searches and moves. 

## Coding 1 (20 points)

We will now design an alpha-beta search agent to play this game. The skeleton functions below follow the pseudocode provided in the lecture and textbook very closely. The inclusion of *k* as an argument allows it to be passed into ```terminal```. Write the main loops of the ```max_value``` and ```min_value``` functions. Remember that the ```result``` function takes in a player argument, and that X is MAX while O is MIN.

In [ ]:
def alpha_beta_search(state, player, k):
  # Initialize a game tree search for (m,n,k) game
  # X is maximizing player, O is minimizing player
  if player == 'X':
    value, move = max_value(state, -float("inf"), float("inf"), k)
  else:
    value, move = min_value(state, -float("inf"), float("inf"), k)
  return value, move

def max_value(state, alpha, beta, k):
  isTerminal, score = terminal(state, k)
  if isTerminal:
    return score, None
  # YOUR CODE HERE
  score = -float("inf")
  act = None
  for a in actions(state):
    old_score = score
    score = max(score, min_value(result(state, 'X', a), alpha, beta, k)[0])
    if old_score < score:
      act = a
    if score >= beta:
      return score, None
    alpha = max(alpha, score)
  return score, act


def min_value(state, alpha, beta, k):
  isTerminal, score = terminal(state, k)
  if isTerminal:
    return score, None
  # YOUR CODE HERE
  score = float("inf")
  act = None
  for a in actions(state):
    old_score = score
    score = min(score, max_value(result(state, 'O', a), alpha, beta, k)[0])
    if old_score > score:
       act = a
    if score <= alpha:
      return score, None
    beta = min(beta, score)
  return score, act

"""
my_state = np.array([['X','O','.'],['X','.','O'],['X','O','.']])
value, move = alpha_beta_search(my_state, 'X', 3)
print(value, move)

my_state = np.array([['X','O','.'],['X','.','O'],['.','.','.']])
value, move = alpha_beta_search(my_state, 'X', 3)
print(value, move)

my_state = np.array([['O','X','.'],['.','O','X'],['.','.','.']])
value, move = alpha_beta_search(my_state, 'O', 3)
print(value, move)
"""

"\nmy_state = np.array([['X','O','.'],['X','.','O'],['X','O','.']])\nvalue, move = alpha_beta_search(my_state, 'X', 3)\nprint(value, move)\n\nmy_state = np.array([['X','O','.'],['X','.','O'],['.','.','.']])\nvalue, move = alpha_beta_search(my_state, 'X', 3)\nprint(value, move)\n\nmy_state = np.array([['O','X','.'],['.','O','X'],['.','.','.']])\nvalue, move = alpha_beta_search(my_state, 'O', 3)\nprint(value, move)\n"

You can test the functions above by either finding the values of terminal states or states close to terminal states, for which you know the winner with certainty. (Remember that states are 2d numpy arrays.) For example, in the (3,3,3) game, ```max_value``` of a state with two X's and a blank in a row should certainly return a move leading to a winning state for X. Same goes for ```min_value``` and O.

Once you're happy with your agent implementation, let's test it out in a full game. The following ```game_loop``` takes in an initial state, the value of *k*, the search algorithm to use (which for now is just ```alpha_beta_search``` and any additional parameters for the search algorithm (none for now). 

When it runs, ```game_loop``` simply goes back and forth between the two players. Each player plans its next move using ```search```, the move is executed, the state is updated, and the opposing player's turn is taken. This ends when we hit a terminal state.

In [ ]:
def game_loop(state, k, search, X_params=[], O_params=[]):
  # Play a (m,n,k) game using provided search function and parameters
  player = 'X'
  isTerminal = False
  while not isTerminal:
    if player == 'X':
      value, move = search(state, player, k, *X_params)
      state = result(state, player, move)
      player = 'O'
    else:
      value, move = search(state, player, k, *O_params)
      state = result(state, player, move)
      player = 'X'
    print(np.matrix(state), "\n")
    isTerminal, _ = terminal(state, k)

  if value > 0: print("X wins!")
  elif value < 0: print("O wins!")
  else: print("Draw!")

Let's have our intelligent agent play against itself! Run the code below to play standard tic-tac-toe. Remember that both players play optimally.

In [ ]:
m, n, k = 3, 3, 3
print((m,n,k))
initial = np.full((m,n), '.')
game_loop(initial, k, alpha_beta_search)

(3, 3, 3)
[['X' '.' '.']
 ['.' '.' '.']
 ['.' '.' '.']] 

[['X' '.' '.']
 ['.' 'O' '.']
 ['.' '.' '.']] 

[['X' 'X' '.']
 ['.' 'O' '.']
 ['.' '.' '.']] 

[['X' 'X' 'O']
 ['.' 'O' '.']
 ['.' '.' '.']] 

[['X' 'X' 'O']
 ['.' 'O' '.']
 ['X' '.' '.']] 

[['X' 'X' 'O']
 ['O' 'O' '.']
 ['X' '.' '.']] 

[['X' 'X' 'O']
 ['O' 'O' 'X']
 ['X' '.' '.']] 

[['X' 'X' 'O']
 ['O' 'O' 'X']
 ['X' 'O' '.']] 

[['X' 'X' 'O']
 ['O' 'O' 'X']
 ['X' 'O' 'X']] 

Draw!


## Response 2 (5 points)

What was the result of the game? Assuming both players play optimally, can we ever get a different result? For example, what if we change the starting player, or choose a different move wherever minimax returns multiple equally good moves? Please show an example of a different output above if you find one.

The game resulted in a draw. Assuming that both players play optimally, it will alwasy be a draw for (m,n,k)=(3,3,3). Changining the starting player will not change this because the playeers will just make the opposite moves. Since the game is discrete, fully observable, and deterministic, choosing an equally good move can not result in better play because there is not ambiguity in what future moves will be avalible. 

Our code is suitable for any values of *m,n,k*. Below are three game instances to try out (you could also try more, although any board larger than 4x4 will probably take an unreasonable amount of time to finish). Run each scenario, think about the results, and answer the questions below.

In [ ]:
# m, n, k = 2, 5, 3
m, n, k  = 3, 4, 3
# m, n, k = 3, 4, 4
print((m,n,k))
initial = np.full((m,n), '.')
game_loop(initial, k, alpha_beta_search)

(3, 4, 3)
[['.' 'X' '.' '.']
 ['.' '.' '.' '.']
 ['.' '.' '.' '.']] 

[['O' 'X' '.' '.']
 ['.' '.' '.' '.']
 ['.' '.' '.' '.']] 

[['O' 'X' 'X' '.']
 ['.' '.' '.' '.']
 ['.' '.' '.' '.']] 

[['O' 'X' 'X' 'O']
 ['.' '.' '.' '.']
 ['.' '.' '.' '.']] 

[['O' 'X' 'X' 'O']
 ['.' 'X' '.' '.']
 ['.' '.' '.' '.']] 

[['O' 'X' 'X' 'O']
 ['O' 'X' '.' '.']
 ['.' '.' '.' '.']] 

[['O' 'X' 'X' 'O']
 ['O' 'X' '.' '.']
 ['X' '.' '.' '.']] 

X wins!


## Response 3 (5 points)

Do you notice anything unusual about the gameplay of some of these scenarios? Pay particular attention to the last game (3,4,4). Why do you think the sequence appears as if neither player is even trying to win?

In the game (m,n,k) = (3,4,4), it is only possible to win with a horizonal 4-in-a-row. Therefore, there is little that a plyaer can do to play strategically becasue the players are consistently needing to horizonally block the other players to ensure that they don't get a horizontal win. They are "playing defensively" the entire game rather than attempting to win. (Hence the XOXO pattern.) This is why it appears as though neither player is attempting to win, becasue they are just attempting to not lose.

The same is true for (m,n,k) = (2,5,3). In fact, this will be true for any game where k > n or k > m, becuase there will only be one direction in which the player can win. Therefore, the players will play defensively as stated above and it will not appear that they are attempting to win and it will result in a draw. 

In the game (m,n,k) = (3,4,3), X wins. The added column gives many move options for 3-in-a-row wins and being the first player, X is the one who gets to capitalize on these options.



In order to expand our agent's capacity to deal with boards larger than (4,4) in a reasonable amount of time, we have to accept some suboptimality. This means cutting off search using depth limiting, which in turn requires an evaluation function for non-terminal states. There is a standard set of strategies for tic-tac-toe which can inform what this looks like, but we'll go with a numerical rather than a rule-based function here.

In [ ]:
def eval(state, k):
  X_indices = [i for i,s in np.ndenumerate(state) if s=='X']
  O_indices = [i for i,s in np.ndenumerate(state) if s=='O']
  blanks = [i for i,s in np.ndenumerate(state) if s=='.']

  X_and_blanks = X_indices + blanks
  Xset = set(X_indices)
  Xbset = set(X_and_blanks)
  X_score = 0
  for i in X_and_blanks:
    for seq in sequences(i, k):
      if seq.issubset(Xbset):
        ratio = len(seq & Xset)/k
        X_score = max(X_score, ratio)

  O_and_blanks = O_indices + blanks
  Oset = set(O_indices)
  Obset = set(O_and_blanks)
  O_score = 0
  for i in O_and_blanks:
    for seq in sequences(i, k):
      if seq.issubset(Obset):
        ratio = len(seq & Oset)/k
        O_score = max(O_score, ratio)
  
  return X_score - O_score

## Response 4 (10 points)

Please briefly describe a high-level interpretation of what this evaluation function is computing and returning. What do the ```X_score``` and ```O_score``` variables represent? As a hint, the conditional in each of the inner loops is triggered whenever there is a combination of *k* X's and blanks (or O's and blanks) in a row. 

X_score and O_score represent the maximum ration of X's (or O's) in a row in the given state. Therefore, the evaluation function is copmputing and returning the difference between how close X is to winning and how close O is to winning (having k-in-a-row). The greater the value, the better that state is for X and the lower the value, the better that state is for O.  This gives us a way of quantifying the state in one number, which we can pass into max_value_depth and min_value_depth as an estimate for the utility of a node.

## Coding 2 (10 points)

We will also need some new functions to implement alpha-beta depth-limited search. They will look very similar to the original functions with some minor changes. First, the search function will take a ```max_depth``` parameter. The two value functions will also take this, along with a current ```depth``` parameter. 

After the terminal check, the value functions should check whether the current depth has hit or exceeded the depth limit. If so, it should return the evaluation of the current state (along with no move as it is treated as a terminal). Complete these functions.

In [ ]:
def alpha_beta_depth_search(state, player, k, max_depth):
  if player == 'X':
    value, move = max_value_depth(state, -float("inf"), float("inf"), k, 1, max_depth)
  else:
    value, move = min_value_depth(state, -float("inf"), float("inf"), k, 1, max_depth)
  return value, move

def max_value_depth(state, alpha, beta, k, depth, max_depth):
  isTerminal, score = terminal(state, k)
  if isTerminal:
    return score, None
  # YOUR CODE HERE
  if depth >= max_depth:
    return eval(state, k), None
  depth += 1
  score = -float("inf")
  act = None
  for a in actions(state):
    old_score = score
    score = max(score, min_value_depth(result(state, 'X', a), alpha, beta, k, depth, max_depth)[0])
    if old_score < score:
      act = a
    if score >= beta:
      return score, None
    alpha = max(alpha, score)
  return score, act  


def min_value_depth(state, alpha, beta, k, depth, max_depth):
  isTerminal, score = terminal(state, k)
  if isTerminal:
    return score, None
  # YOUR CODE HERE
  if depth >= max_depth:
    return eval(state, k), None
  depth += 1
  score = float("inf")
  act = None
  for a in actions(state):
    old_score = score
    score = min(score, max_value_depth(result(state, 'O', a), alpha, beta, k, depth, max_depth)[0])
    if old_score > score:
       act = a
    if score <= alpha:
      return score, None
    beta = min(beta, score)
  return score, act


We can now try our hand a larger boards, provided that we sufficiently limit the search depth. Of course, we now lose any guarantee of optimality, and we may get different results when using different depths. We have three games below calling ```alpha_beta_depth_search```. Search depths of 5 are the default max depths, but you'll be trying a few other combinations when answering the following questions.

In [ ]:
#m, n, k = 4, 4, 3
#m, n, k = 4, 4, 4
m, n, k = 5, 5, 4
print((m,n,k))
initial = np.full((m,n), '.')
max_depth_X = 6
max_depth_O = 5
game_loop(initial, k, alpha_beta_depth_search, [max_depth_X], [max_depth_O])

(5, 5, 4)
[['X' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.']] 

[['X' 'O' '.' '.' '.']
 ['.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.']] 

[['X' 'O' '.' 'X' '.']
 ['.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.']] 

[['X' 'O' 'O' 'X' '.']
 ['.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.']] 

[['X' 'O' 'O' 'X' '.']
 ['X' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.']] 

[['X' 'O' 'O' 'X' '.']
 ['X' '.' 'O' '.' '.']
 ['.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.']] 

[['X' 'O' 'O' 'X' '.']
 ['X' '.' 'O' '.' '.']
 ['X' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.']] 

[['X' 'O' 'O' 'X' '.']
 ['X' '.' 'O' '.' '.']
 ['X' '.' '.' '.' '.']
 ['O' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.']] 

[['X' 'O' 'O' 'X' '.']
 ['X' '.' 'O' '.' '.']


## Response 5 (15 points)

Please answer each of the questions below.
1.   Try using a small depth (3 to 5) and a large depth (6 or more) for the (4,4,3) game (keep both players identical). Do you notice any differences in the results? Look at the sequence of moves, not just the outcome.
2.   Up to what depth did you find our agents to play (4,4,4) in a reasonable amount of time? Is the outcome always the same?
3.   You should see that both players draw the (5,5,4) game when they play at depth 5. But what happens when you increase the depth of the X player to 6 while keeping O at 5? Why?



ENTER YOUR RESPONSES HERE
1. X wins in both cases but the sequence of moves is different. For example, the first move is different with depth 3 and depth 6. X wins in fewer moves with the greater depth because it has access to more poosible outcomes and therefore can make more efficient moves. 
2. depth = 6 was the run with the largest depth that executed in a reasonable time. With this depth, it took about 10 seconds. With depth = 7, it took over a minute to execute. The outcomes were always the same for the different depths. 
3. When we increase the X depth to 6, X wins. This may be because the larger depth gives X an advantage since X has more information and can therefore make more informed, optimal moves.